# The dockstring regression benchmark

In this tutorial we evaluate a dummy linear regression model with the dockstring regression benchmark.
The goal of the notebook is to demonstrate how to correctly run the benchmark.

Specifically, we use the following simple model: $$f_\theta(\text{mol}) = \theta \cdot \phi_{\text{atom count}}(\text{mol})$$
where $\phi_{\text{atom count}}$ is a vector containing the count of each atom type (carbon, oxygen, nitrogen, hydrogen, etc).

As expected, the performance is **very bad**: this is because the docking score is a complex function
that depends on much more than just the atom count (unlike simpler properties like the logP score).

In [72]:
# Import everything- install missing libraries as needed
# (everything here is quite standard)

from pathlib import Path

# import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import sklearn.metrics

from rdkit import Chem

## 1. Downloading the dataset

If you haven't downloaded the dataset,
run the following lines of code in your terminal (from the repository root directory!)
to download the dataset.
These lines of code are also given dockstring's README.

```bash
mkdir -p datasets
wget -O datasets/dockstring-dataset.tsv https://figshare.com/ndownloader/files/30562257?private_link=95f2fed733dec170b998
```

Also, download the dataset splits:

# TODO!!! CLEAN THIS UP!!



In [9]:
dataset_path = Path("../datasets/dockstring-dataset.tsv")
assert dataset_path.exists()  # Throw an error if the dataset is missing!

dataset_split_path = Path("../datasets/cluster_split.tsv")
assert dataset_split_path.exists()

## 2. Loading the dataset with pandas

We read in the dataset and the train/test split file to make the train and test datasets.

You can copy/paste the code from this section to use with your method.

In [15]:
df = pd.read_csv(dataset_path, sep="\t").set_index("inchikey")  # since our dataset is tab-delimited
df

,smiles,PPARD,ABL1,ADAM17,ADRB1,ADRB2,AKT2,MAOB,CASP3,DHFR,...,EGFR,F10,GBA,MAPK1,MAPK14,PLK1,SRC,THRB,F2,KDR
inchikey,,,,,,,,,,,,,,,,,,,,,
UMVWYQXKBPJMOF-UHFFFAOYNA-N,C1=C(C2=C(C=C1O)OC(C(C2=O)=O)C3=CC=C(C(=C3)O)O)O,-8.2,-9.1,-9.0,-9.3,-9.7,-8.7,-8.4,-7.2,-8.9,...,-9.1,-8.4,-9.1,-9.3,-8.3,-9.0,-8.4,-8.8,-8.2,-8.0
NGOGFTYYXHNFQH-UHFFFAOYNA-N,O=S(=O)(N1CCNCCC1)C2=CC=CC=3C2=CC=NC3,-7.1,-9.5,-7.0,-7.6,-7.7,-8.2,-6.3,-6.1,-8.4,...,-7.5,-6.6,-8.0,-8.3,-6.9,-8.6,-7.7,-8.1,-6.8,-7.4
BGVLELSCIHASRV-QPEQYQDCNA-N,C=1C=C2S/C(/N(CC)C2=CC1OC)=C\C(=O)C,-6.6,-7.4,-5.9,-7.0,-7.0,-7.1,-6.9,-5.5,-7.0,...,-6.6,-5.9,-6.4,-7.1,-6.3,-7.0,-6.2,-7.9,-5.8,-6.2
KTUFNOKKBVMGRW-RPGFEBOUNA-N,C=1(N=C(C=2C=NC=CC2)C=CN1)NC=3C=C(NC(C4=CC=C(C...,-10.8,-10.2,-11.0,-10.4,-11.5,-11.0,-3.3,-9.4,-10.1,...,-11.0,-9.0,-9.3,-10.2,-8.8,-10.4,-9.5,-7.4,-9.7,-11.4
LLJRXVHJOJRCSM-UHFFFAOYNA-N,C1=CC=2C(=CNC2C=C1)C=3C=CN=CC3,-7.7,-8.6,-7.4,-8.4,-8.2,-7.7,-7.9,-6.2,-8.1,...,-8.0,-7.0,-7.7,-8.0,-7.8,-7.9,-6.8,-8.5,-6.6,-7.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DWPVTFJKTGBFRK-UHFFFAOYNA-N,ClC1=CC(S(=O)(=O)N2N=NC=3C2=CC=CC3)=C(OCC)C=C1,-7.5,-9.0,-7.8,-8.1,-8.4,-7.8,-7.4,-6.9,-9.2,...,-8.0,-7.8,-8.2,-8.3,-7.3,-8.2,-7.6,-8.7,-7.8,-7.8
GZKYOKRGPTXXIJ-YAQRNVERNA-N,ClC=1C=CC(C(=O)NN=C2CCCC2)=CC1,-8.1,-8.6,-8.0,-8.1,-8.1,-7.3,-8.5,-6.3,-8.3,...,-7.9,-7.1,-8.0,-7.6,-7.0,-8.2,-7.0,-8.3,-7.2,-8.2
SFJOYBYSPNCEKG-UHFFFAOYNA-N,O=C1N(C(=O)N(C2=C3N(C(=C21)C4=CC=CC=C4)C=5C(N=...,-8.9,-10.3,-9.6,-11.5,-7.7,-8.9,-6.3,-8.9,-8.9,...,-10.5,-9.5,-8.7,-8.8,-8.2,-10.9,-9.1,-6.3,-9.2,-7.5


In [16]:
splits = (
    pd.read_csv(dataset_split_path, sep="\t")
    .set_index("inchikey")  # use same index as dataset
    .loc[df.index]  # re-order to match the dataset
)
splits

,smiles,cluster,split
inchikey,,,
UMVWYQXKBPJMOF-UHFFFAOYNA-N,C1=C(C2=C(C=C1O)OC(C(C2=O)=O)C3=CC=C(C(=C3)O)O)O,0,train
NGOGFTYYXHNFQH-UHFFFAOYNA-N,O=S(=O)(N1CCNCCC1)C2=CC=CC=3C2=CC=NC3,1,train
BGVLELSCIHASRV-QPEQYQDCNA-N,C=1C=C2S/C(/N(CC)C2=CC1OC)=C\C(=O)C,2,train
KTUFNOKKBVMGRW-RPGFEBOUNA-N,C=1(N=C(C=2C=NC=CC2)C=CN1)NC=3C=C(NC(C4=CC=C(C...,3,train
LLJRXVHJOJRCSM-UHFFFAOYNA-N,C1=CC=2C(=CNC2C=C1)C=3C=CN=CC3,4,train
...,...,...,...
DWPVTFJKTGBFRK-UHFFFAOYNA-N,ClC1=CC(S(=O)(=O)N2N=NC=3C2=CC=CC3)=C(OCC)C=C1,19021,test
GZKYOKRGPTXXIJ-YAQRNVERNA-N,ClC=1C=CC(C(=O)NN=C2CCCC2)=CC1,11206,train
SFJOYBYSPNCEKG-UHFFFAOYNA-N,O=C1N(C(=O)N(C2=C3N(C(=C21)C4=CC=CC=C4)C=5C(N=...,5968,train


In [17]:
df_train = df[splits["split"] == "train"]
df_test = df[splits["split"] == "test"]

## 3. Featurize the data

Here, our features are just the atom counts, so we write a function to calculate this.
However, these features will probably not be sufficient for a real model,
so you will need to replace this to do well on this benchmark.

In [32]:
from typing import List
def smiles_to_atom_counts(smiles: str) -> List[int]:
    
    # List the atom types which we are counting explicitly
    atom_list = ["C", "O", "H", "S", "N", "F", "Cl", "Br", "I", "P"]
    atom_to_idx = {symbol: i for i, symbol in enumerate(atom_list)}
    
    # Make the mol object (including implicit hydrogens)
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    
    # Count the atoms
    atom_counts = [0 for _ in atom_list] + [0]  # extra atom
    for atom in mol.GetAtoms():
        # if it isn't in the dictionary, it is counted
        # as an "extra" atom at the end
        idx = atom_to_idx.get(atom.GetSymbol(), -1)  
        atom_counts[idx] += 1
    
    return atom_counts

In [44]:
# Make X_train and X_test as floats
X_train = np.asarray(list(map(smiles_to_atom_counts, df_train["smiles"])), dtype=float)
X_test = np.asarray(list(map(smiles_to_atom_counts, df_test["smiles"])), dtype=float)

Preview the arrays to make sure they look correct...

In [45]:
X_train

array([[15.,  7., 10., ...,  0.,  0.,  0.],
       [14.,  2., 17., ...,  0.,  0.,  0.],
       [13.,  2., 15., ...,  0.,  0.,  0.],
       ...,
       [28.,  2., 22., ...,  0.,  0.,  0.],
       [19.,  3., 17., ...,  0.,  0.,  0.],
       [15.,  2., 16., ...,  0.,  0.,  0.]])

In [46]:
X_train.shape, X_test.shape

((221274, 11), (38881, 11))

## 4. Train the models and get test predictions

Here, we train on the 5 targets that we suggest in our paper.

In [71]:
target_list = ["ESR2", "F2", "KIT", "PARP1", "PGR",]
name_to_pred = dict()
for target_name in target_list:
    
    # 1: data preparation
    # ================================
    
    # Get the train and test scores for this target
    y_train = df_train[target_name].values
    y_test = df_test[target_name].values

    # Positive docking scores are unphysical, so we clip all docking scores to a maximum of 5.0
    y_train = np.minimum(y_train, 5.0)
    y_test = np.minimum(y_test, 5.0)

    # Some points in the training data have a score of NaN!
    # So, we filter these points out
    y_train_nonan = y_train[~np.isnan(y_train)]
    X_train_nonan = X_train[~np.isnan(y_train)]

    y_test_nonan = y_test[~np.isnan(y_test)]
    X_test_nonan = X_test[~np.isnan(y_test)]

    # 2: fitting the model and getting predictions
    # ============================
    model = LinearRegression()
    model.fit(X_train_nonan, y_train_nonan)
    y_test_pred = model.predict(X_test_nonan)
    
    # Store results
    name_to_pred[target_name] = (y_test_nonan, y_test_pred)

## 5. Evaluate results

The evaluation metric that we suggest is the test set $R^2$ score.

The results here are better than just guessing the mean (which would give $R^2=0$),
but are worse than any method that we report in our paper (as one might expect!)

Hopefully, if you adapt this code for your method, you will do better!!

In [76]:
for target_name, (y_test_nonan, y_test_pred) in name_to_pred.items():
    r2_score = sklearn.metrics.r2_score(y_true=y_test_nonan, y_pred=y_test_pred)
    print(f"{target_name+':':<6s} {r2_score:.3g}")

ESR2:  0.314
F2:    0.619
KIT:   0.427
PARP1: 0.655
PGR:   0.121
